<a href="https://colab.research.google.com/github/marekhennig/175IC-machine-learning/blob/main/Zad_Wyklad_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install aiohttp

!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

import concurrent.futures
import requests
import threading
import asyncio
import aiohttp
import multiprocessing
import time
import shutil
import os
from urllib import request
import timeit
from concurrent.futures import ThreadPoolExecutor

images = []
response = requests.get("https://picsum.photos/v2/list")
res = response.json()
for info in res:
    images.append(info['download_url'])


def download_image(link):
    filename = link.split('/')[4]
    fileformat = 'jpg'
    r = requests.get(link, stream=True)
    if r.status_code == 200:
        r.raw.decode_content = True
        with open(filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)
    else:
        request.urlretrieve(link, "downloads/{}.{}".format(filename, fileformat))
        print("{}.{} downloaded ".format(filename, fileformat))


def download_album():
    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(download_image, images)


def main():
    download_album()


if __name__ == "__main__":
    print("Time taken to download images using multithreading: {}".format(
        timeit.Timer(main).timeit(number=1)))


def main_multi():
    pool = multiprocessing.Pool(multiprocessing.cpu_count())
    result = pool.map(download_image, [image for image in images])


if __name__ == "__main__":
    print("Time taken to download images using multiprocessing: {}".format(timeit.Timer(main).timeit(number=1)))


async def download_image_asyncio(link, session):
    filename = link.split('/')[4]
    fileformat = 'jpg'
    async with session.get(link) as response:
        with open("{}.{}".format(filename, fileformat), 'wb') as fd:
            async for data in response.content.iter_chunked(1024):
                fd.write(data)


async def main():
    async with aiohttp.ClientSession() as session:
        tasks = [download_image_asyncio(image, session) for image in images]
        return await asyncio.gather(*tasks)

if __name__ == "__main__":
    start_time = timeit.default_timer()
    loop = asyncio.get_event_loop()
    results = loop.run_until_complete(main())
    time_taken = timeit.default_timer() - start_time

    print("Time taken to download images using AsyncIO: {}".format(time_taken))

Time taken to download images using multithreading: 2.2562578759999496
Time taken to download images using multiprocessing: 2.0645176339999125
Time taken to download images using AsyncIO: 0.6717623220000633
